# Stock Price Predictor with ML

This code uses Technical Anlaysis Indicators to predict the price of a stock one week in the future.

The program:
* Takes the list of tickers in the "tickers" variable.
* Downloads historical price data for each
* Calculates Technical Indicators
* Trains a Machine Learning model
* Then uses the model to predict the price of each stock five trading days ahead
* Displays a list of the top 10 best perfomers

## Libraries:

In [4]:
!pip install pandas_ta
!sed -i 's/from numpy import NaN as npNaN/from numpy import nan as npNaN/g' /usr/local/lib/python3.11/dist-packages/pandas_ta/momentum/squeeze_pro.py
import pandas_ta as ta
import yfinance as yf
import pandas as pd
import numpy as np
import time

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from numpy import nan as npNaN
import statsmodels.api as sm


## Ticker List:

In [58]:
#tickers to be processed:
tickers = [ "VIST", "TSLA", "MSTR", "SPY", "BABA", "ANF", "HMY", "PLTR", "TCOM", "NVDA",
    "KO", "INTC", "NU", "EWZ", "QQQ", "XLE", "PBR", "GOOGL", "GOLD", "BIOX",
    "HUT", "IBIT", "COIN", "MRNA", "MELI", "HPQ", "NIO", "GPRK", "GLD", "BRKB",
    "AAPL", "JD", "AMD", "VALE", "DIA", "AMZN", "GLOB", "LMT", "IWM", "META",
    "TGT", "BAK", "FXI", "AVGO", "MSFT", "MU", "JMIA", "ARKK", "PAGS", "ETHA",
    "NKE", "BBD", "SH", "RIOT", "PHG", "SATL", "AAL", "ORCL", "XLF", "ADBE",
    "ABEV", "OXY", "UPST", "BIDU", "XOM", "UBER", "C", "TSM", "QCOM", "BITF",
    "WMT", "GM", "PFE", "PAAS", "CAT", "X", "NEM", "SAN", "F", "T", "JPM",
    "CVX", "BK", "LAC", "BA", "UAL", "DISN", "GGB", "PEP", "V", "JNJ",
    "EBAY", "ERJ", "MMM", "XYZ", "PYPL", "LLY", "AMAT", "ROST", "SNOW",
    "UNH", "MO", "MRK", "AZN", "PM", "LVS", "RIO", "URBN", "TXR", "WFC",
    "SHEL", "RBLX", "HAL", "TEN", "MCD", "WBA", "SUZ", "HSBC", "AMGN", "DOW",
    "SAP", "ZM", "SBUX", "XP", "CRM", "ADI", "PG", "MRVL", "GS", "TM",
    "BB", "HWM", "IBM", "VZ", "TXN", "MOS", "AXP", "TRIP", "EEM", "PSX",
    "GILD", "UL", "HOG", "MDT", "IEUR", "NFLX", "FSLR", "LAR", "ABT", "ABBV",
    "SPOT", "CL", "VEA", "SHOP", "BHP", "CAAP", "STNE", "CDE", "ITUB", "XLP",
    "SID", "YY", "HD", "AIG", "HSY", "GE", "IVE", "COST", "FMX", "MDLZ",
    "GLW", "LYG", "BBAS3", "MA", "BP", "BBV", "TIMB", "DOCU", "TTE", "BSBR",
    "NG", "STLA", "ADGO", "IVW", "NGG", "ABNB", "SNAP", "XLRE", "AEM", "SE",
    "XLK", "CSCO", "DESP", "KMB", "DEO", "XLV", "KGC", "LRCX", "ARCO", "SPGI",
    "SONY", "MUX", "AAP", "BIIB", "KOFM", "EA", "PETR3", "ACN", "HDB", "BMY",
    "PANW", "DE", "SPCE", "BKNG", "ORLY", "FDX", "PBI", "SYY", "ARM", "GFI",
    "BRFS", "HL", "MSI", "CVS", "PCAR", "ERIC", "VOD", "NUE", "IBB", "VRSN",
    "FCX", "HMC", "NVS", "ABEV3", "GSK", "XLY", "ING", "XROX", "SLB", "NTES",
    "CX", "TMUS", "TEFO", "XLC", "AMX", "PKS", "BNG", "RTX", "LND", "RACE",
    "EFX", "BCS", "ETSY", "WBO", "XLI", "DAL", "EBR", "MUFG", "NOKA", "HON",
    "SDA", "GT", "TMO", "CAR", "YELP"]

In [ ]:
#complete list of CEDEARS tickers for reference
tikers2 = [ "VIST", "TSLA", "MSTR", "SPY", "BABA", "ANF", "HMY", "PLTR", "TCOM", "NVDA",
    "KO", "INTC", "NU", "EWZ", "QQQ", "XLE", "PBR", "GOOGL", "GOLD", "BIOX",
    "HUT", "IBIT", "COIN", "MRNA", "MELI", "HPQ", "NIO", "GPRK", "GLD", "BRKB",
    "AAPL", "JD", "AMD", "VALE", "DIA", "AMZN", "GLOB", "LMT", "IWM", "META",
    "TGT", "BAK", "FXI", "AVGO", "MSFT", "MU", "JMIA", "ARKK", "PAGS", "ETHA",
    "NKE", "BBD", "SH", "RIOT", "PHG", "SATL", "AAL", "ORCL", "XLF", "ADBE",
    "ABEV", "OXY", "UPST", "BIDU", "XOM", "UBER", "C", "TSM", "QCOM", "BITF",
    "WMT", "GM", "PFE", "PAAS", "CAT", "X", "NEM", "SAN", "F", "T", "JPM",
    "CVX", "BK", "LAC", "BA", "UAL", "DISN", "GGB", "PEP", "V", "JNJ",
    "EBAY", "ERJ", "MMM", "XYZ", "PYPL", "LLY", "AMAT", "ROST", "SNOW",
    "UNH", "MO", "MRK", "AZN", "PM", "LVS", "RIO", "URBN", "TXR", "WFC",
    "SHEL", "RBLX", "HAL", "TEN", "MCD", "WBA", "SUZ", "HSBC", "AMGN", "DOW",
    "SAP", "ZM", "SBUX", "XP", "CRM", "ADI", "PG", "MRVL", "GS", "TM",
    "BB", "HWM", "IBM", "VZ", "TXN", "MOS", "AXP", "TRIP", "EEM", "PSX",
    "GILD", "UL", "HOG", "MDT", "IEUR", "NFLX", "FSLR", "LAR", "ABT", "ABBV",
    "SPOT", "CL", "VEA", "SHOP", "BHP", "CAAP", "STNE", "CDE", "ITUB", "XLP",
    "SID", "YY", "HD", "AIG", "HSY", "GE", "IVE", "COST", "FMX", "MDLZ",
    "GLW", "LYG", "BBAS3", "MA", "BP", "BBV", "TIMB", "DOCU", "TTE", "BSBR",
    "NG", "STLA", "ADGO", "IVW", "NGG", "ABNB", "SNAP", "XLRE", "AEM", "SE",
    "XLK", "CSCO", "DESP", "KMB", "DEO", "XLV", "KGC", "LRCX", "ARCO", "SPGI",
    "SONY", "MUX", "AAP", "BIIB", "KOFM", "EA", "PETR3", "ACN", "HDB", "BMY",
    "PANW", "DE", "SPCE", "BKNG", "ORLY", "FDX", "PBI", "SYY", "ARM", "GFI",
    "BRFS", "HL", "MSI", "CVS", "PCAR", "ERIC", "VOD", "NUE", "IBB", "VRSN",
    "FCX", "HMC", "NVS", "ABEV3", "GSK", "XLY", "ING", "XROX", "SLB", "NTES",
    "CX", "TMUS", "TEFO", "XLC", "AMX", "PKS", "BNG", "RTX", "LND", "RACE",
    "EFX", "BCS", "ETSY", "WBO", "XLI", "DAL", "EBR", "MUFG", "NOKA", "HON",
    "SDA", "GT", "TMO", "CAR", "YELP"]

## Main Algorithm:

In [61]:
report_data = {
      'Ticker': [],
      'Latest Close Value':[],
      'Predicted Value':[],
      'Expected Return':[],
      'Precision R2':[]
      }

for i in range(0, len(tickers)):
  try:
    time.sleep(1)
    print('\nDownloading ticker: ' + tickers[i] + ' (' + str(i+1) + '/' + str(len(tickers)) + ')')
    df = yf.download(tickers[i], interval='1d', start="2019-01-01", progress= False)
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

    df.columns = df.columns.droplevel(1) #Drop second level column (ticker name)

    df['Next_Week_Close'] = df['Close'].shift(-5)

    # Simple Moving Average (SMA): Average price over the last 50 periods
    df['SMA_50'] = ta.sma(df['Close'], length=50)

    # Exponential Moving Average (EMA): Weighted average that reacts faster to recent price changes, using 50 periods
    df['EMA_50'] = ta.ema(df['Close'], length=50)

    # Relative Strength Index (RSI): Momentum indicator that measures the magnitude of recent price changes to evaluate overbought/oversold conditions, using a 14-period lookback
    df['RSI'] = ta.rsi(df['Close'], length=14)

    # Moving Average Convergence Divergence (MACD): Trend-following momentum indicator, using 12 and 26 periods for the fast and slow EMAs and a 9-period signal line
    macd_df = ta.macd(df['Close'])
    # Add MACD values to your original DataFrame
    df['MACD'] = macd_df['MACD_12_26_9']        # MACD line
    df['MACD_Signal'] = macd_df['MACDs_12_26_9'] # Signal line
    df['MACD_Histogram'] = macd_df['MACDh_12_26_9'] # MACD Histogram

    # Bollinger Bands: Volatility indicator using a 20-period moving average and 2 standard deviations
    bollinger = ta.bbands(df['Close'], length=20, std=2)
    df['BB_Upper'] = bollinger['BBU_20_2.0']  # Upper Bollinger Band
    df['BB_Middle'] = bollinger['BBM_20_2.0'] # Middle Band (20-period SMA)
    df['BB_Lower'] = bollinger['BBL_20_2.0']  # Lower Bollinger Band

    # Stochastic Oscillator: Momentum indicator comparing closing prices to price ranges over 14 periods with a 3-period %D moving average
    stoch = ta.stoch(df['High'], df['Low'], df['Close'], k=14, d=3)
    df['%K'] = stoch['STOCHk_14_3_3'] # %K line (main line)
    df['%D'] = stoch['STOCHd_14_3_3'] # %D line (3-period moving average of %K)

    # Average True Range (ATR): Volatility indicator measuring the average range of price movement over the last 14 periods
    df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)

    # Drop rows with missing values due to shifting and indicator calculation
    df.dropna(inplace=True)

    scaler = MinMaxScaler()

    X = df.drop(columns=['Next_Week_Close','Open', 'High', 'Low'])
    y = df['Next_Week_Close']

    X = scaler.fit_transform(X)


    modelSVR2 = SVR(C=2.0, degree=2, kernel='linear')
    modelSVR2.fit(X, y)
    y_test_pred2 = modelSVR2.predict(X)

    r2_2 = r2_score(y, y_test_pred2)

    latest_prediction = modelSVR2.predict(X[-1].reshape(1, -1))
    last_close_price = df['Close'].iloc[-1]
    expected_return = (latest_prediction[0] - last_close_price) / last_close_price

    report_data['Ticker'].append(tickers[i])
    report_data['Latest Close Value'].append(last_close_price)
    report_data['Expected Return'].append(expected_return * 100)
    report_data['Predicted Value'].append(latest_prediction[0])
    report_data['Precision R2'].append(r2_2 * 100)

  except Exception as e:
            print(f"Error when loading ticker {i}: {e}")


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRKB']: YFTzMissingError('possibly delisted; no timezone found')


Error when loading ticker 29: 'NoneType' object is not subscriptable












<ipython-input-61-133390cc2969>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_Week_Close'] = df['Close'].shift(-5)
<ipython-input-61-133390cc2969>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SMA_50'] = ta.sma(df['Close'], length=50)
<ipython-input-61-133390cc2969>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

ERROR:yfinance:Could not get exchangeTimezoneName for ticker 'BBD' reason: 'chart'
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BBD']: YFTzMissingError('possibly delisted; no timezone found')


Error when loading ticker 51: 'NoneType' object is not subscriptable





































...


Error when loading ticker 245: 'NoneType' object is not subscriptable


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BNG']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-01-01 -> 2025-03-21)')



Error when loading ticker 246: 'NoneType' object is not subscriptable







Error when loading ticker 253: 'NoneType' object is not subscriptable






ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['NOKA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-01-01 -> 2025-03-21)')



Error when loading ticker 258: 'NoneType' object is not subscriptable








## Show Report:

In [62]:
report_data = pd.DataFrame(report_data).sort_values(by = 'Expected Return', ascending= False).reset_index(drop= True)
report_data.head(10).round(2)

,Ticker,Latest Close Value,Predicted Value,Expected Return,Precision R2
0,ADGO,0.00,0.09,86369.62,-2.37
1,MRNA,34.62,37.32,7.80,95.30
2,SDA,4.66,4.91,5.47,43.71
3,LLY,801.65,839.75,4.75,99.31
4,PAGS,7.81,8.12,4.02,97.77
5,COST,903.92,939.91,3.98,99.17
6,ETHA,14.64,15.21,3.87,83.73
7,GS,541.41,560.89,3.60,98.45
8,AVGO,194.95,201.15,3.18,98.48
9,EBR,7.03,7.23,2.86,89.09


In [54]:
report_data.tail(10).round(2)

,Ticker,Latest Close Value,Predicted Value,Expected Return,Precision R2
10,VIST,49.23,49.17,-0.13,99.27
11,KO,69.16,68.78,-0.55,97.24
12,PLTR,86.24,85.67,-0.66,95.74
13,BABA,141.10,140.16,-0.67,98.00
14,HMY,12.11,11.98,-1.06,96.79
15,ANF,79.79,78.93,-1.08,98.55
16,BIOX,4.63,4.57,-1.33,96.29
17,NU,11.73,11.39,-2.91,96.92
18,INTC,24.05,23.29,-3.16,95.50
19,MSTR,297.49,282.63,-5.00,96.16
